In [ ]:
import json

class Inventory:
    def __init__(self):
        self.items = []

    def add_item(self, item):
        self.items.append(item)

    def display_items(self):
        if not self.items:
            print("Inventory is empty.")
        else:
            print("Inventory:")
            for i, item in enumerate(self.items, start=1):
                print(f"{i}. {item.name}")
                
class Upgrade:
    def __init__(self, name, price):
        self.name = name
        self.price = price

class Seed:
    def __init__(self, name, growth_time,value, price):
        self.name = name
        self.growth_time = growth_time
        self.watered = False
        self.grown = False
        self.value=value
        self.price=price

    def water(self):
        self.watered = True

    def grow(self):
        self.grown = True

class Plot:
    def __init__(self):
        self.seed = None

    def plant_seed(self, seed):
        if self.seed is None:
            self.seed = seed
            print(f"Planted {seed.name} seed on this plot.")
        else:
            print("Plot already has a seed planted.")

    def water_seed(self):
        if self.seed:
            self.seed.water()
            print("Seed watered.")
        else:
            print("No seed planted on this plot.")

    def grow_seed(self):
        print(f"{self.seed.name} has grown!")

class Farm:
    def __init__(self):
        self.plots = [Plot()]
        self.wallet=0
        self.upgrade_inventory = Inventory()  
        self.seed_inventory = Inventory()     
        self.autowater=0

    def add_plot(self):
        self.plots.append(Plot())
        print("Added a new plot to your farm.")

    def display_plots(self):
        print("Your farm:")
        for i, plot in enumerate(self.plots):
            print(f"Plot {i+1}: {plot.seed.name if plot.seed else 'Empty'}")
            
    def displaywallet(self):
        print(f"Wallet Balance: {self.wallet}")

    def buy_seed(self, chosen_seed):
        self.seed_inventory.add_item(chosen_seed)  
        self.wallet -= chosen_seed.value
        print(f"{chosen_seed.name} seed purchased successfully!")
        
    def shop(self):
        print("Welcome to the shop!",f"Wallet Balance: {self.wallet}")
        print("1. Buy Autowater Upgrade (Cost: 100 coins)\n2. Buy a new plot (Cost: ",50*len(self.plots),"coins)\n3. Buy Seeds")
        choice = input("Enter your choice: ")
        if choice == "1":
            if self.wallet >= 100:  
                autowater_upgrade = Upgrade("Autowater", 100)
                self.buy_upgrade(autowater_upgrade)
            else:
                print("Not enough coins to buy Autowater upgrade.")

        if choice=="2":
            if self.wallet>= 50*len(self.plots):
                self.add_plot()
            else:
                print("Not enough coins.")
        elif choice == "3":
            self.seedsprices()  
            choice = input("Enter the number of the seed you want to buy or 'q' to quit: ")
            if choice.lower() != 'q':
                try:
                    choice = int(choice)
                    if 0 <= choice < len(seeds):
                        chosen_seed = seeds[choice]
                        if chosen_seed not in self.seed_inventory.items:  
                            if self.wallet >= chosen_seed.value:
                                self.buy_seed(chosen_seed)  
                            else:
                                print("Not enough coins to buy this seed.")
                        else:
                            print("You already have this seed.")
                    else:
                        print("Invalid seed choice.")
                except ValueError:
                    print("Invalid input. Please enter a number.")

    def buy_upgrade(self, upgrade):
        self.upgrade_inventory.add_item(upgrade)  
        self.wallet -= upgrade.price
        print(f"{upgrade.name} upgrade purchased successfully!")
        
    def get_upgrade(self, upgrade_name):
        for upgrade in self.upgrade_inventory.items:
            if upgrade.name == upgrade_name:
                return upgrade
        return None

    def pass_day(self):
        print("A day passes on the farm.")
        autowater_upgrade = self.get_upgrade("Autowater")
        for plot in self.plots:
            if plot.seed:
                if autowater_upgrade:
                    plot.seed.watered = True
                if plot.seed.watered:  
                    if autowater_upgrade is None:
                        plot.seed.watered = False  
                    plot.seed.growth_time -= 1  
                    if plot.seed.growth_time <= 0:  
                        plot.grow_seed() 
                        self.wallet += plot.seed.value
                        plot.seed = None  
                    else:
                        print(f"{plot.seed.name} is growing. Remaining growth time: {plot.seed.growth_time} days.")
                else:
                    print(f"{plot.seed.name} needs to be watered to grow.")
                    
    def load_seeds(self, filename):
        seeds = []
        try:
            with open(filename, 'r') as file:
                data = json.load(file)
                for seed_data in data:
                    name = seed_data['name']
                    growth_time = seed_data['growth_time']
                    value = seed_data['value']
                    price = seed_data['price']
                    seed = Seed(name, growth_time, value, price)
                    seeds.append(seed)
        except FileNotFoundError:
            print("Seed file not found.")
        return seeds
    

    def displayseeds(self):
        print("Available seeds:")
        self.seed_inventory.display_items()  

        while True:
            choice = input("Enter the number of the seed you want to plant or 'q' to quit: ")
            if choice.lower() == 'q':
                break
            try:
                choice = int(choice)
                chosen_seed = self.seed_inventory.items[choice - 1]  
                self.display_plots()
                plot_number = int(input("Enter the plot number where you want to plant the seed: "))
                plot_number -= 1
                try:
                    self.plots[plot_number].plant_seed(chosen_seed)
                    break
                except IndexError:
                    print("There is no plot there")
            except (ValueError, IndexError):
                print("Invalid input. Please enter a valid number.")
        
    def seedsprices(self):
        seeds = self.load_seeds("seeds.json")  
        print("Available seeds with prices to buy:")
        for i, seed in enumerate(seeds, start=0):
            print(f"{i}. {seed.name} (Price to buy: {seed.price} coins, Growth Time: {seed.growth_time}, Crop Value:{seed.value})")



Day=0
            
farm = Farm()

farm.display_plots()

seeds = farm.load_seeds("seeds.json")
print("\nWelcome to the Farm!")
while True:
    print("Day:",Day)
    print("1. Plant seeds")
    print("2. Water Plot")
    print("3. Pass Day")
    print("4. Go Shopping")
    print("5. Check Inventory")
    print("6. Exit")
    choice=input("What will you do? :")

    if choice =="1":
        print("Will you plant? ")
        farm.displayseeds() 

        
    elif choice=="2":
        farm.display_plots()
        watering=int(input("Which plot? :"))
        watering-=1
        try:
            farm.plots[watering].water_seed()
        except:
            print("There is no plot here.")
    elif choice=="3":
        farm.pass_day()
        Day+=1


    elif choice=="4":
        farm.shop()
        
    elif choice=="5":
        inventnum=input("Check your: 1. Seeds, 2. Upgrades, or 3. Plots")
        if inventnum=="1":
            farm.seed_inventory.display_items() 
        elif inventnum=="2":
            farm.upgrade_inventory.display_items()  
        elif inventnum=="3":
            farm.display_plots()
        else:
            print("Invalid choice pick 1-3")
         
    elif choice=="6":
        print("Adios!")
        break
    else:
        print("Invalid choose 1-6")
